In [ ]:
import wandb
wandb.login()

In [ ]:
import numpy as np
import pandas as pd
import torch.nn as nn

import pickle
import torch
import time
import copy
import sys
import time
import os
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [ ]:
import yaml
config_wbd = {}
with open('/home/quanhhh/Documents/model/model_spec/sweep.yml', 'r') as f:
    config_wdb = yaml.safe_load(f)

In [ ]:
config_wdb

In [ ]:
#dataloader
# import numpy as np
# import pickle
# from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, config, path):
        super().__init__()
        self.config = config
        self.path = path
        self.load_dataset()
    
    def __len__(self):
        return self.data_.shape[0]
    
    def __getitem__(self, index):
        return {'input':self.data_[index], 
                'labels': self.labels[index]}
    
    def load_dataset(self):
        data = {}
        path_ = self.config['data_dir'] + self.path
        with np.load(path_, 'rb') as f:
            data['data'] = f['data']
            data['label'] = f['label']
        self.data_ = data['data'].transpose(0,2,1)
        self.data_ = np.expand_dims(self.data_, axis=1)
        
        
        self.labels = data['label']
    def getshape(self):
        return self.data_.shape[1:]
            

# Model

In [ ]:
#model
# import torch.nn as nn
class Conv_gru(nn.Module):
    def __init__(self, shape_data, num_filters, kernel_size, maxpooling, d_hidden, num_layers, dropout=0.1):
        super().__init__()
        self.shape_data = shape_data
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        # self.strides = strides
        self.maxpooling = maxpooling
        self.d_hidden = d_hidden
        self.num_layers = num_layers
        self.dropout = dropout

        # input -> conv2d -> batchnorm -> maxpool2d ->conv2d -> batchnorm -> maxpool 
        
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

        self.conv1 = nn.Conv2d(
                            in_channels=self.shape_data[0], 
                            out_channels=self.num_filters[0], 
                            kernel_size=self.kernel_size[0],
                            )

        self.conv2 = nn.Conv2d(
                            in_channels=self.num_filters[0], 
                            out_channels=self.num_filters[1], 
                            kernel_size=self.kernel_size[1],
                            )

        self.f_activation1 = nn.ReLU()
        self.f_activation2 = nn.ReLU()


        # torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
        self.batchNorm1 = nn.BatchNorm2d(num_features=self.num_filters[0])
        self.batchNorm2 = nn.BatchNorm2d(num_features=self.num_filters[1])

        # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        
        self.maxpooling1 = nn.MaxPool2d(kernel_size=self.maxpooling[0])
        self.maxpooling2 = nn.MaxPool2d(kernel_size=self.maxpooling[1])

    
        self.h_conv1 = int((self.shape_data[1] - self.kernel_size[0]) + 1)
        self.w_conv1 = int((self.shape_data[2] - self.kernel_size[0]) + 1)

        self.h_maxpool1 = int((self.h_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1)
        self.w_maxpool1 = int((self.w_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1)  

        self.h_conv2 = int((self.h_maxpool1 - self.kernel_size[1]) + 1)
        self.w_conv2 = int((self.w_maxpool1 - self.kernel_size[1]) + 1)

        self.h_maxpool2 = int((self.h_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1)
        self.w_maxpool2 = int((self.w_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1)  

        self.gru = nn.GRU(self.w_maxpool2, self.d_hidden, self.num_layers, batch_first=True, dropout=self.dropout)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        
        #layer1
        self.conv1_out = self.conv1(x)
        self.conv1relu_out = self.f_activation1(self.conv1_out)
        
        #layer2
        
        self.batchNorm1_out = self.batchNorm1(self.conv1relu_out)
        self.maxpooling1_out = self.maxpooling1(self.batchNorm1_out)
        
        self.conv2_out = self.conv2(self.maxpooling1_out)
        self.conv2relu_out = self.f_activation2(self.conv2_out)
        
        #layer3
        self.batchNorm2_out = self.batchNorm2(self.conv2relu_out)
        self.maxpooling2_out = self.maxpooling2(self.batchNorm2_out)
        # self.outcnn = self.conv3(self.maxpooling2_out)

        # # 
        # self.outcnn = torch.squeeze(self.outcnn, dim=1) 
        self.outcnn = torch.sum(self.maxpooling2_out, dim=1)/self.num_filters[1] #(batch, h, w)
        # self.outcnn = torch.flatten(self.maxpooling2_out, start_dim=1, end_dim=2)
        
        #layer4
        h_c = self.init_hidden(len(self.outcnn))
        self.out, _ = self.gru(self.outcnn, h_c)
        return self.softmax(self.out[:, -1, :])
            
    def init_hidden(self, batch_size):
        self.batch = batch_size
        weight = next(self.gru.parameters()).data
        
        hidden = weight.new(self.num_layers, batch_size, self.d_hidden).zero_()        
        # cell = weight.new(self.num_layers, batch_size, self.d_hidden).zero_()
        
        return hidden #, cell)
    # def get_shape(self):
    #     print('Input: [{}, {}, {}]'.format(
    #         self.batch,
    #         self.shape_data[0],
    #         self.shape_data[1],
    #     )
    #     )
    #     print('Conv1|Batch_norm1: [{}, {}, {}, {}] | Maxpool1: [{}, {}, {}, {}]'.format(
    #         self.batch,
    #         self.num_filters[0],
    #         self.h_conv1,
    #         self.w_conv1,
    #         self.batch,
    #         self.num_filters[0],
    #         self.h_maxpool1,
    #         self.w_maxpool1)
    #         )
    #     print('Conv2|Batch_norm1: [{}, {}, {}, {}] | Maxpool2: [{}, {}, {}, {}]'.format(
    #         self.batch,
    #         self.num_filters[1],
    #         self.h_conv2,
    #         self.w_conv2,
    #         self.batch,
    #         self.num_filters[0],
    #         self.h_maxpool2,
    #         self.w_maxpool2)
    #         )
    #     print('Custom layer: [{}, {}, {}]'.format(
    #         self.batch,
    #         self.h_maxpool2,
    #         self.w_maxpool2)
    #         )
    #     print('Output: [{}, {}]'.format(
    #         self.batch,
    #         self.d_hidden)
    #         )
        

        

# Train

In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from torch import optim
# import time
# import copy

class ClassificationTrainer():
    def __init__(self, model, train_data, val_data, val_set, device, config, buzz='buzz1'):
        self.model = model
    
        self.train_data = train_data
        self.val_data = val_data
        self.val_set = val_set
        self.device = device
        self.config = config
        self.max_acc = 95.67
        self.max_val = 0
        self.train_loss_list = list()
        self.test_loss_list = list()
        self.train_acc_list = list()
        self.test_acc_list = list()
        self.best_epoch = 0
        self.best_model = None
        self.buzz = buzz
        self.path = self.config['result_dir'] + "{}_model_{}num_layers_({},{})filters_({},{})kernel_size_({},{})maxpool_{}lr_{}epoch_{}batch_size.pt".format(
            self.buzz,
            self.config['num_layers'],
            self.config['num_filters_1'],
            self.config['num_filters_2'],
            self.config["kernel_size_1"],
            self.config["kernel_size_2"],
            # self.config["stride_1"],
            # self.config["stride_2"],
            self.config["maxpool_1"],
            self.config["maxpool_2"],
            self.config['lr'],
            self.config['num_epoch'],
            self.config['batch_size']
        )
    
    def train_epoch(self, criterion, opt, epoch):
        train_loss = 0.0
        train_acc = 0.0
        self.model.train()
        lenght_train = 0
        for i, data in enumerate(self.train_data):
            input = data['input'].float().to(self.device)            
            target = data['labels'].float().to(self.device)
            out = self.model(input)
            lenght_train += len(input)
            opt.zero_grad()
            loss = criterion(out, target)
            loss.backward()
            opt.step()
            train_loss += loss.item()
            # print('\rEpoch: {}\t| Train_loss: {:.6f}\t|Train accuracy: {:.6f}'.format(epoch, train_loss/lenght, train_acc/lenght), end='')
        
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.train_data):
                input = data['input'].float().to(self.device)            
                target = data['labels'].float().to(self.device)
                
                out = self.model(input)
                
                classes = torch.argmax(out, axis=1)
                target_ = torch.argmax(target, axis=1)
                #print(classes, target_)
                train_acc += torch.sum(classes == target_)
        self.train_loss = train_loss/lenght_train
        self.train_acc = train_acc.item()*100/lenght_train
        self.train_loss_list.append(self.train_loss)
        self.train_acc_list.append(self.train_acc)
        
        
        # print('\nEpoch: {}\t| Train_loss: {:.6f}\t|Train accuracy: {:.6f}'.format(epoch, train_loss, train_acc))
        if self.val_data is None:
            if train_loss < self.min_loss:
                self.min_loss = train_loss
                self.best_epoch = epoch
        else:
            self.validate_epoch(criterion, epoch)
        
    def validate_epoch(self, criterion, epoch):
        test_loss = 0.0
        test_acc = 0.
        val_acc = 0.0
        lenght = 0
        lenghtval = 0
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_data):
                input = data['input'].float().to(self.device)
                target = data['labels'].float().to(self.device)
                
                out = self.model(input)
                lenght += len(input)
                classes = torch.argmax(out, axis=1)
                target_ = torch.argmax(target, axis=1)
                test_acc += torch.sum(classes == target_)
                loss = criterion(out, target)
                test_loss += loss.item()
            
            for i, data in enumerate(self.val_set):
                input = data['input'].float().to(self.device)
                target = data['labels'].float().to(self.device)
                
                out = self.model(input)
                lenghtval += len(input)
                
                classes = torch.argmax(out, axis=1)
                target_ = torch.argmax(target, axis=1)

                val_acc += torch.sum(classes == target_)
            
        val_acc = val_acc*100/lenghtval  
        test_loss = test_loss/lenght
        test_acc = test_acc.item()*100/lenght
        self.test_loss_list.append(test_loss)
        self.test_acc_list.append(test_acc)
        print('\rEpoch: {}\t| Train_loss: {:.6f}\t|Train accuracy: {:.6f}\t| Test_loss: {:.6f}\t|Test accuracy: {:.6f}\t|Val accuracy: {:.6f} '.format(
                                                                                                                                        epoch, 
                                                                                                                                        self.train_loss, 
                                                                                                                                        self.train_acc, 
                                                                                                                                        test_loss, 
                                                                                                                                        test_acc, 
                                                                                                                                        val_acc), 
                                                                                                                                        end="\r") 
        wandb.log({'train_loss': self.train_loss, 'test_loss': test_loss})
        if val_acc >= self.max_val:
            self.max_val = val_acc
            self.best_epoch = epoch
            if self.max_val >= 97.6:
                self.best_model_path = self.config['result_dir'] + "{}_model_{}num_layers_({},{})filters_({},{})kernel_size_({},{})maxpool_{}lr_{}epoch_{}batch_size.pt".format(
                    self.buzz,
                    self.config['num_layers'],
                    self.config['num_filters_1'],
                    self.config['num_filters_2'],
                    self.config["kernel_size_1"],
                    self.config["kernel_size_2"],
                    # self.config["stride_1"],
                    # self.config["stride_2"],
                    self.config["maxpool_1"],
                    self.config["maxpool_2"],
                    self.config['lr'],
                    epoch,
                    self.config['batch_size']
                )
                self.best_model = copy.deepcopy(self.model)
            # torch.save(self.model.state_dict(), path)
    def train(self):
        self.model.to(self.device)  
        
        start = time.perf_counter()
        model_opt = optim.Adam(self.model.parameters(), self.config['lr'], weight_decay = self.config['weight_decay'])
        criterion = nn.CrossEntropyLoss()

        print("-----START TRAINING CLASSIFICATION-----")
        for epoch in range(1, self.config['num_epoch'] + 1):
            self.train_epoch(criterion, model_opt, epoch)  
        print('\n')
        self.config['train_time'] = (time.perf_counter() - start)/60
        print("----COMPLETED TRAINING CLASSIFICATION-----")

        self.config["best_epoch"] = self.best_epoch
        if self.best_model is not None:
            torch.save(self.best_model.state_dict(), self.best_model_path)
            self.config['best_model_path'] = self.best_model_path
        else:
            self.config['best_model_path'] = ''

        self.config['val_acc'] = self.max_val
        self.config['train_loss'] = self.train_loss_list
        self.config['train_acc'] = self.train_acc_list
        self.config['test_loss'] = self.test_loss_list
        self.config['test_accuracy'] = self.test_acc_list
        # self.config['model_path'] = self.path 
        # self.save_loss()

    def get_updated_config(self):
        return self.config
    
    def load_model(self, path = None):
        if path is None:
            path = self.config['result_dir'] + "model.pt"
        self.model.load_state_dict(torch.load(path))
        self.model.eval()        
      

# Train.py

In [ ]:
sweep_id = wandb.sweep(sweep=config_wdb, project='quanh')
# sweep_id = "9g9inm8n"

In [ ]:
import torch
import sys
import time
import numpy as np
import pandas as pd

from torch.utils.data.dataloader import DataLoader


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

def trainwandb():
    with wandb.init(config=config_wbd):
        config = wandb.config
        # train_set = CustomDataset(config, 'buzz1_traindata.npz')
        # shape = train_set.getshape()
        # data_dataloader = DataLoader(train_set, 
        #                               batch_size = 128,
        #                               shuffle = True)
        # train_dataloader = []
        # test_dataloader = []
        # for i, batch in enumerate(data_dataloader):
        #     if i < len(data_dataloader)*0.7:
        #         train_dataloader.append(batch)
        #     else:
        #         test_dataloader.append(batch)

        #buzz2
        train_set = CustomDataset(config, 'buzz2_vn_traindata.npz')
        train_dataloader = DataLoader(train_set, 
                                    batch_size = 128,
                                    shuffle = True)
        shape = train_set.getshape()
        val_set = CustomDataset(config, 'buzz2_vn_testdata.npz')
        test_dataloader = DataLoader(val_set,
                                    batch_size = 128,
                                    shuffle = True)
        ###
        val_set = CustomDataset(config, 'buzz2_vn_valdata.npz')
        val_set_ = DataLoader(val_set,
                                    batch_size = 128,
                                    shuffle = True)
        torch.manual_seed(10)
        classification_model = Conv_gru(shape_data=shape, 
                            num_filters=(config['num_filters_1'], config['num_filters_2']),
                            kernel_size=(config['kernel_size_1'], config['kernel_size_2']), 
                            # strides=(config['stride_1'], config['stride_2']), 
                            maxpooling=(config['maxpool_1'], config['maxpool_2']), 
                            d_hidden=config['d_hidden'], 
                            num_layers= config['num_layers'])
        classification_model.float()
        classification_trainer = ClassificationTrainer(model=classification_model,
                                                    train_data=train_dataloader,
                                                    val_data=test_dataloader,
                                                    val_set=val_set_,
                                                    device=device,
                                                    config=config,
                                                    buzz="buzz2_vn")
        classification_trainer.train()
        config = classification_trainer.get_updated_config()
        val_acc = config['val_acc']
        wandb.log({'val_acc': val_acc})
        
        if val_acc >= 97.6:
            wandb.alert(
                title=f"MaiAnh found new high accuracy ID {sweep_id}", 
                text=f"Accuracy {val_acc} "
            )
#         filename = "{}num_layers_({},{})kernel_size_{}n_filters_({},{})stride_({},{})maxpool_{}lr_{}batch_size".format(
#             config["num_layers"],
#             config["kernel_size_1"],
#             config["kernel_size_2"],
#             config["num_filters"],
#             config["stride_1"],
#             config["stride_2"],
#             config["maxpool_1"],
#             config["maxpool_2"],
#             config['lr'],
#             config['batch_size']
#         ).replace(".", "_")
#         PATH = os.path.join("configs_wandb/", "{}.yml".format(filename))
#         with open(PATH, 'w') as outfile:
#             yaml.dump(config, outfile, default_flow_style=False)
        print('ok')

In [ ]:
wandb.agent(sweep_id, function=trainwandb, count=200)